In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import pickle
with open('data.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [3]:
train = df[df['Date'] < '2015-01-01']
test = df[df['Date'] > '2014-12-31']

In [4]:
cv = CountVectorizer(stop_words='english')

X_fit = cv.fit(train['text'])
X_transform = X_fit.transform(train['text'])
X_test = X_fit.transform(test['text'])
y_test = test['Label']
df_train = pd.DataFrame(X_transform.toarray(), columns=cv.get_feature_names())
y = train['Label']

In [5]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=5, n_iter=100, random_state=42)
svd_fit = svd.fit(X_transform)  
svd_transform = svd_fit.transform(X_transform)
svd_transform_test = svd_fit.transform(X_test)

In [6]:
df_train_svd = pd.DataFrame(svd_transform)

In [7]:
df_test_svd = pd.DataFrame(svd_transform_test)

In [8]:
df_norm_train = (df_train_svd - df_train_svd.mean()) / (df_train_svd.max() - df_train_svd.min())
# df_norm_train

In [9]:
df_norm_test = (df_test_svd - df_test_svd.mean()) / (df_test_svd.max() - df_test_svd.min())
# df_norm_test

In [10]:
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

gnb = GaussianNB().fit(df_norm_train, y) 
gnb_predictions = gnb.predict(df_norm_test) 
  
# accuracy on X_test 
# accuracy = gnb.score(X_te, y_te) 
# print (accuracy) 
  
# # creating a confusion matrix 
# cm = confusion_matrix(y_te, gnb_predictions) 
# cm

In [11]:
pd.crosstab(y_test,gnb_predictions, colnames=["Predicted"], rownames=["Actual"] )

Predicted,0,1
Actual,,
0,9,177
1,10,182
